In [121]:
import sys
import yaml
import itertools
import subprocess

In [122]:
from pathlib import Path

# Compile original Lyra2 implementations

Need to import python functions that compile Lyra from a git submodule. Manipulate path as per https://stackoverflow.com/a/29747054/1269892

In [123]:
sys.path.append(str(Path('../lyra/Lyra2/tests').resolve()))

In [124]:
from harness import build_lyra2, unlist_values
from harness import compose_sponge_name, compose_lyra2_name

In [125]:
with open('harness.yml', 'r') as config:
    params = yaml.load(config)

In [126]:
build_lyra2(params)

In [127]:
build_path0 = Path('./bin42').resolve()
if not build_path0.exists():
    print('Could not find ./bin42, directory for original executables')

# Compile ported Java implementation

In [128]:
subprocess.run(['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'])

CompletedProcess(args=['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'], returncode=0)

In [129]:
build_path1 = Path('./target').resolve()
if not build_path1.exists():
    print('Could not ./target, directory for compiled executables')

# Execution observer

In [130]:
import time

In [131]:
from statistics import median

In [132]:
class ProcessObserver:
    def __init__(self, ntimes=3):
        self.ntimes = ntimes

    def run(self, *args):
        times = [-1 for i in range(self.ntimes)]
        for i in range(self.ntimes):
            fst = time.time()
            
            process = subprocess.run(*args)
                        
            lst = time.time()
            
            if process.returncode != 0:
                print(args[0] + ' failed to run')

                continue
            
            times[i] = lst - fst
            
        return [median(times)]

# Compare the implementations

In [133]:
password, salt = 'password', 'salt'

In [134]:
klens = [1, 10, 100]

In [135]:
tcosts = [1, 10, 100]

In [136]:
mcosts = [3, 10, 100] # there must be at least 3 rows in the memory matrix

In [137]:
observer = ProcessObserver(ntimes=1)

In [138]:
time_data0 = {}
time_data1 = {}

for matrix in unlist_values(params['matrix']):
    option = matrix['option']

    threads = matrix['threads']
    columns = matrix['columns']

    sponge = matrix['sponge']
    rounds = matrix['rounds']
    blocks = matrix['blocks']

    bench = matrix['bench']

    [sponge, _] = compose_sponge_name(sponge)

    name = compose_lyra2_name(
        option, threads, columns, sponge, rounds, blocks
    )
    
    executable0 = build_path0.joinpath(name)
    
    time_data0[name] = {}
    
    for klen, tcost, mcost in itertools.product(klens, tcosts, mcosts):
        fst = time.time()
        
        verdict = observer.run([executable0, password, salt, str(klen), str(tcost), str(mcost)])
    
        time_data0[name][klen, tcost, mcost] = verdict[0]

In [139]:
print(time_data0)

{'lyra2-generic-x86-64-threads-1-columns-256-sponge-blake2b-rounds-1-blocks-8': {(1, 1, 3): 0.003759622573852539, (1, 1, 10): 0.002137422561645508, (1, 1, 100): 0.0035758018493652344, (1, 10, 3): 0.0072193145751953125, (1, 10, 10): 0.002790689468383789, (1, 10, 100): 0.011165380477905273, (1, 100, 3): 0.00498652458190918, (1, 100, 10): 0.00822305679321289, (1, 100, 100): 0.0665900707244873, (10, 1, 3): 0.0019631385803222656, (10, 1, 10): 0.0019085407257080078, (10, 1, 100): 0.003332376480102539, (10, 10, 3): 0.002104043960571289, (10, 10, 10): 0.0026426315307617188, (10, 10, 100): 0.00918436050415039, (10, 100, 3): 0.0037245750427246094, (10, 100, 10): 0.008621692657470703, (10, 100, 100): 0.06557989120483398, (100, 1, 3): 0.0022203922271728516, (100, 1, 10): 0.001833200454711914, (100, 1, 100): 0.003403902053833008, (100, 10, 3): 0.0020399093627929688, (100, 10, 10): 0.003084421157836914, (100, 10, 100): 0.009123802185058594, (100, 100, 3): 0.0036988258361816406, (100, 100, 10): 0.008

In [140]:
executable1 = build_path1.joinpath('lyra2-1.2-SNAPSHOT-jar-with-dependencies.jar')